This notebook includes the calculation of volumetric grid charges and the optimization of empirical household energy consumption with dynamic energy prices for this case.

### Requirements

Load the necessary packages to complete the optimization.

In [1]:
import pandas as pd
import numpy as np
from gurobipy import *

### Methods

The first method is used in the optimization to determine the start and end points of each charging session, as well as the overall charged energy within one session. The second method is used to write the results of the optimization into a dataframe, enabling further analysis.

In [2]:
#Function to calculate start point, end point, and total energy charged for each charging session
def chargPerUser(load,percent):
    
    #List to store information about charging sessions
    charging_sessions = []
    
    total_energy_charged = 0
    b=False
    b1=False

    #Iterate through all time steps
    for i in range(len(percent)):
        
        #If the plugin hour is not a full hour
        if percent[i] < 1 and percent[i] != 0 and b == False:
            total_energy_charged += load[i] #Add charged energy
            start_index = i #Save the start point
            b = True
            
        #If the plugin hour is a full hour   
        elif percent[i]==1 and percent[i-1]==0:
            total_energy_charged += load[i]
            start_index = i
            b = True
        
        #Full charging hour between plugin and plugout
        elif percent[i] == 1:
            total_energy_charged += load[i]
            b = True
            b1 = True
            
        #Plugout after hours of full charging and the end hour is not a full hour   
        elif percent[i] < 1 and percent[i] != 0 and b == True and b1 == True and percent[i+1] == 0:
            total_energy_charged += load[i]
            end_index = i #Save the end point
            charging_sessions.append({'start_index': start_index, 'end_index': end_index, 'total_energy': total_energy_charged})
            total_energy_charged = 0
            b = False
            b1 = False
            
        #Plugout after no hour of full charging and the end hour is not a full hour 
        elif percent[i] < 1 and percent[i] != 0 and b == True and b1 == False and percent[i+1] == 0:
            total_energy_charged += load[i]
            end_index = i
            charging_sessions.append({'start_index': start_index, 'end_index': end_index, 'total_energy': total_energy_charged})
            total_energy_charged = 0
            b = False
            
        #Plugout in the same hour as plugin
        elif percent[i] == 0 and b == True and b1 == False:
            end_index = start_index
            charging_sessions.append({'start_index': start_index, 'end_index': end_index, 'total_energy': total_energy_charged})
            total_energy_charged = 0
            b = False
        
        #Plugout after hours of full charging and the end hour is a full hour 
        elif percent[i] == 0 and b == True and b1 == True:
            end_index = i-1
            charging_sessions.append({'start_index': start_index, 'end_index': end_index, 'total_energy': total_energy_charged})
            total_energy_charged = 0 
            b = False
            b1 = False
        
        #If no energy is charged or if several not full hours follow each other
        else:
            total_energy_charged += load[i]
            
    return(charging_sessions)

In [3]:
#Write resluts of optimization in dataframe
def get_results_in_df(variableNames, n_timesteps, model): #  "def name(argumente)" zeigt den Anfang einer Funktion an, die ich später beliebig aufrufen kann
   

    results_df = pd.DataFrame(columns = variableNames, index = [t for t in range(n_timesteps)] )   # Hier wird ein leeres DataFrame erstellt, dass als Spalteneinträge die Namen der Gutobi-variablen hat und als Zeilen die Zeitschritte der Entscheidungsvariablen

    for n in variableNames:                                         #Iteration über alle Zielvariablen
        for t in range(n_timesteps):                                #Iteration über alle Zeitschritte
            VarName = n + f"[{t}]"                                  #Hier wird ein String erstellt, der die Form n[t] hat. Mit diesem wird später die Zielvariable ausgelesen
            try:                                                    #try - except ist eine hilfreiche Methdoe, um Fehler abzufangen. Wenn in der nächsten Zeile ein error passiert, bspw. wegen eines Schreibfehlers in meinen zielvariablen, wird die Funktion nicht abgebrochen sondern weiter ausgeführt.
                results_df.loc[t][n] =  model.getVarByName(VarName).x    #Auslesen der Zielvaribale
            except:
                pass
            
    return results_df

### Dictionaries

The following dictionaries are used to reference column names by indexing:

In [4]:
#Dictionary for households
hh_dic={1:"SFH3_HH",2:"SFH4_HH",3:"SFH5_HH",4:"SFH7_HH",5:"SFH9_HH",6:"SFH12_HH",7:"SFH14_HH",8:"SFH16_HH",9:"SFH18_HH",10:"SFH19_HH",
        11:"SFH20_HH",12:"SFH21_HH",13:"SFH22_HH",14:"SFH27_HH",15:"SFH28_HH",16:"SFH29_HH",17:"SFH30_HH",18:"SFH32_HH",19:"SFH34_HH",
        20:"SFH36_HH",21:"SFH38_HH",22:"SFH39_HH"}

In [5]:
#Dictionary for heatpumps
hp_dic={1:"SFH3_HP",2:"SFH4_HP",3:"SFH5_HP",4:"SFH7_HP",5:"SFH9_HP",6:"SFH12_HP",7:"SFH14_HP",8:"SFH16_HP",9:"SFH18_HP",10:"SFH19_HP",
        11:"SFH20_HP",12:"SFH21_HP",13:"SFH22_HP",14:"SFH27_HP",15:"SFH28_HP",16:"SFH29_HP",17:"SFH30_HP",18:"SFH32_HP",19:"SFH34_HP",
        20:"SFH36_HP",21:"SFH38_HP",22:"SFH39_HP"}

### Optimization

The following method describes the optimization of empirical household consumption with volumetric grid charges to minimize total costs. HH consumption remains inflexible. Shifting HP consumption is restricted by a maximum number of blocked time steps (sigma_max) in each interval of length 'length'. The total consumption over an interval must match the empirical consumption within that interval. EV shifting is freely possible between plug-in and plug-out times, as long as the maximum charging power (ev_max) of the wallbox is not surpassed and the charged energy for the session matches the empirical data.

In [6]:
#Optimization of one household with hp flexibility within intervals and flexible ev between plugin and plugout with dynamic energy prices
def optimized_HP_EV(hh, hp, ev_user, peak_ev, flexibility, part, p_flex, p_grid):
    
    l = len(hh)
    l_ev= len(ev_user.columns)//2
    ev_max = peak_ev
    hp_max = max(hp)
    sigma_max = flexibility
    part = part
    
    #1. create Model
    model  = Model("simpleHP")
        

    #2. define Variables
    
    #2.1 hp load over all time steps
    d_hp = model.addVars(l, vtype=GRB.CONTINUOUS, lb=0, ub=hp_max, name="d_hp")
    
    #2.2 procentual hp shifts over all time steps
    sigma = model.addVars(l,vtype=GRB.BINARY, name="sigma")
    
    #2.3 ev charging load over all time steps
    d_ev = model.addVars(l, l_ev, vtype=GRB.CONTINUOUS, lb=0, ub=ev_max, name="d_ev")
    
    #2.4 total consumption over all time steps
    d = model.addVars(l, vtype=GRB.CONTINUOUS, lb=0, name="d")

    #2.5 total ev consupmtion per time step
    #ev = model.addVars(l, vtype=GRB.CONTINUOUS, lb=0, name="ev")
    

    #3. constraints
    
    #3.1 total consumption over all time steps as sum of hp, hh and all evs
    model.addConstrs(d[t] == d_hp[t]+hh[t]+quicksum(d_ev[t,u] for u in range(l_ev)) for t in range(l))
    
    #HP
    #3.2 calculation of hp load
    model.addConstrs(d_hp[t] >= (1-sigma[t])*hp[t] for t in range(l))
    
    #3.3 hp flexibility is restricted to flex_hours through binary variable sigma
    model.addConstrs(quicksum(sigma[k] for k in range(part*i,part*i+part)) <= sigma_max for i in range(l//part))

    #3.4 hp load over time part must at least be empirical hp
    model.addConstrs(quicksum(d_hp[k] for k in range(part*i,part*i+part)) >= quicksum(hp[k] for k in range(part*i,part*i+part)) for i in range(l//part))

    
    #EV
    #iterate through all evs (users) that charge at this garage
    for u in range(l_ev):
        ev = ev_user.iloc[:,u*2]
        ev_perc = ev_user.iloc[:,u*2+1]
        
        #3.5 in every timestep ev power demand must be smaller than ev_max
        model.addConstrs(d_ev[t,u] <= ev_perc[t]*ev_max for t in range(l))
        
        #get for every charging session start and end time and the total charged energy
        charging_sessions=chargPerUser(ev,ev_perc)
        list_length = len(charging_sessions)
        
        #iterate through all charging sessions of one user
        for i in range (list_length):
            start_index = charging_sessions[i]['start_index']
            end_index = charging_sessions[i]['end_index']
            total_energy = charging_sessions[i]['total_energy']
            
            #3.6 the charged power between start and end must be equal to the total charged energy
            model.addConstr(quicksum(d_ev[t,u] for t in range(start_index,end_index+1)) == total_energy)

            
    #4. optimize
    objective_expr = quicksum(d[t] * (p_flex.iloc[t,0]+p_grid) for t in range(l))
    
    #4.1 set the costs as the variable to be minimized
    model.setObjective(objective_expr,GRB.MINIMIZE)

    #4.2 perform optimization
    model.optimize()
    
    #5 return results
    #5.1 get objective value
    cost = model.getObjective().getValue()
    
    #get values for total load d for every hour
    var =["d","d_hp"]
    load_after_shift = get_results_in_df(var,l, model)

    #save values of charging station load in every time step
    ev_df = pd.DataFrame(index=range(l))

    for k in range(l_ev):
        col_name = f'ev_{k}'
        ev_df[col_name] = [d_ev[t, k].x for t in range(l)]
    
    return cost, load_after_shift, ev_df

### Read in data

Read in EV charging data, household and heat pump consumption patterns, maximum charging power of each wallbox, day-ahead market prices, garage IDs as well as the total load in the empirical case. All data has been pre-processed beforehand.

In [7]:
ev_hourly = pd.read_csv('Hourly_EV_Charging.csv')
hh_hp = pd.read_csv('Hourly_HH_HP.csv')
peak_ev = pd.read_csv('Min_Peak_Load.csv')
p_flex = pd.read_csv('Hourly_Flexible_Prices.csv')
unique_garage = pd.read_csv('Unique_Garage.csv')
load_before_shift = pd.read_csv('load_before_shift.csv',index_col=0)

Calculate total costs for each household in the baseline scenario with volumetric grid charges and a flat electricity tariff. The combined cost of the grid charges and the electricity tariff is 30.85 cents per kWh.

In [8]:
cost_before_shift_flat = []
for i in range(len(peak_ev)): #iterate over all HHs
    cost = 0
    for k in range(len(p_flex)): #iterate over all time steps
        cost += load_before_shift.iloc[k,i]*0.3085
    cost_before_shift_flat.append(cost)

Calculate total costs with dynamic electricity prices but without grid charges. Subtract these costs from the costs of the baseline scenario, and then determine the price per kWh to obtain the amount for the volumetric grid charges.

In [9]:
cost_before_shift_p_flex = []
for i in range(len(peak_ev)):
    cost = 0
    for k in range(len(p_flex)):
        cost += load_before_shift.iloc[k,i]*p_flex.iloc[k,0]
    cost_before_shift_p_flex.append(cost)

In [10]:
p_grid_total=sum(cost_before_shift_flat)-sum(cost_before_shift_p_flex)
load_total =sum(load_before_shift.sum())
p_grid = p_grid_total/load_total
p_grid= round(p_grid * 10000) / 10000

### Perform optimization

Perform optimization for each HH for dynamic energy prices, to determine the new household consumption patterns.

In [11]:
load_after_shift = pd.DataFrame()
cost_after_shift = []
hp_after_shift = pd.DataFrame()
data = np.zeros((len(hh_hp.index),len(peak_ev))) 
ev_after_shift = pd.DataFrame(data)
kappa = 4 #interval length
sigma_max = 2 #number of blocked time steps allowed in one interval

#Iterate over all households
for i in range(len(peak_ev)):
    
    #Select all users per garage
    selected_columns = ev_hourly.filter(regex=f'^{unique_garage.iloc[i,0]}-', axis=1)
    
    #Perform optimization
    cost, load, ev_df = optimized_HP_EV(hh_hp[hh_dic[i+1]],hh_hp[hp_dic[i+1]], selected_columns, peak_ev.iloc[i,0],sigma_max,kappa,p_flex,p_grid)

    #Calculate ev load after shift
    for k in range(len(ev_df)):
        sum=0
        for j in range(len(ev_df.columns)):
            sum += ev_df.iloc[k,j]
        ev_after_shift.iloc[k,i] = sum
    
        
    #Save total costs
    cost_after_shift.append(cost)
    
    #Store shifted loads per household
    load_after_shift[f"{hh_dic[i+1]}_d"] = load.iloc[:,0]
    hp_after_shift[f"{hh_dic[i+1]}_d"] = load.iloc[:,1]

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-31
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 52565 rows, 57024 columns and 118818 nonzeros
Model fingerprint: 0xf1be6167
Variable types: 47520 continuous, 9504 integer (9504 binary)
Coefficient statistics:
  Matrix range     [1e-02, 6e+00]
  Objective range  [2e-01, 4e-01]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e-02, 4e+01]
Found heuristic solution: objective 2873.9701928
Presolve removed 52564 rows and 56952 columns
Presolve time: 1.90s
Presolved: 1 rows, 72 columns, 72 nonzeros
Found heuristic solution: objective 2809.5910150
Variable types: 72 continuous, 0 integer (0 binary)

Root relaxation: objective 2.809106e+03, 1 iterations, 0.00 seconds (0.

#### Save Data

In [12]:
#save as a dataframe
cost_before_shift_df = pd.DataFrame(cost_before_shift_flat, columns=['cost_before_shift'])
cost_after_shift_df = pd.DataFrame(cost_after_shift, columns=['cost_after_shift'])

#ev_after_shift.to_csv('ev_after_shift_4kappa.csv')
#hp_after_shift.to_csv('hp_after_shift_4kappa.csv')
cost_before_shift_df.to_csv('cost_before_shift.csv')
cost_after_shift_df.to_csv('cost_after_shift.csv')
load_after_shift.to_csv('load_after_shift.csv')